# Introduction
This notebook is to showcase some basic MongoDB queries.

# 1. Basic setup

In [ ]:
# Import libraries
from pprint import PrettyPrinter

import pandas as pd
from pymongo import MongoClient
import pymongo

In [ ]:
# Set up client to use MongoDB Atlas
client = MongoClient("mongodb+srv://<username>:<password>@cluster0.at2qasm.mongodb.net/?retryWrites=true&w=majority")

In [ ]:
pp = PrettyPrinter(indent=2)
pp.pprint(list(client.list_databases()))

In [ ]:
# Assign database Sample Restaurants to db
db = client["sample_restaurants"]

In [ ]:
# Print list of collections in database
for c in db.list_collections():
    print(c["name"])

In [ ]:
# Assign collection "restaurant" to collection
collection = db["restaurants"]

In [ ]:
# Count number of documents within collection
collection.count_documents({})

In [ ]:
# Print the first document
result = collection.find_one({})
pp.pprint(result)

In [ ]:
# 1. Find all documents, but only show some of the key values
result = collection.find({},
    projection={"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}
)
pp.pprint(list(result))


# Find and Aggregate

According to some articles, the differences between \$find and \$aggregate are:
- \$find takes less time to process than \$aggregate
- However, $aggregate works like a pipeline, through different stages, and allow more advanced queries

It seems aggregate is more recommended for most works

More on this in:
- https://studio3t.com/knowledge-base/articles/mongodb-aggregation-framework/
- https://www.digitalocean.com/community/tutorials/how-to-use-aggregations-in-mongodb

In [ ]:
# 2. Write a MongoDB query to find the restaurants which have score between 30 and 39
# in Bronx and serve Pizza

result = collection.aggregate([
    {"$match":{"grades": {"$elemMatch":{"score":{"$gt":30 , "$lt":39}}}, "borough": "Bronx", "cuisine": "Pizza"}},
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}}
])
pp.pprint(list(result))

In [ ]:
# 3. Write a MongoDB query to find the restaurants which do not prepare any cuisine of 'American' 
# and achieved a score more than 70 and located in the longitude less than -65.754168.
result = collection.aggregate([
    {"$match":
        {
            "cuisine": {"$ne" :"American"}, 
            "grades.score": {"$gt": 70}, 
            "address.coord": {"$lt": -65.754168}
        }
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0, "grades":1}},
])
pp.pprint(list(result))

# df= pd.DataFrame(result).set_index("name")
# df

In [ ]:
# 4. Write a MongoDB query to find the restaurants which do not prepare any cuisine of 'American ' 
# and achieved a grade point 'A' not belongs to the borough Brooklyn. 
# The document must be displayed according to the cuisine in descending order
result = collection.aggregate([
    {"$match":
        {
            "cuisine": {"$ne" :"American"}, 
            "grades.grade": "A",
            "borough": {"$not":{"$eq":"Brooklyn"}}
        }
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1,"grades.grade":1, "_id":0}},
    {"$sort" : {"cuisine": 1}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("cuisine")
df

# $regex

\$regex is a function for text-search

There are 3 types of syntax, however, my recommendation is: <br>
`{ <field>: { $regex: 'pattern', $options: '<options>' } }` <br>

For example:<br>
`{"name": {"\$regex": "^Wil", "$options": "i"}}`

[Parameters of $option](https://www.mongodb.com/docs/manual/reference/operator/query/regex/#mongodb-query-op.-options)

A few characters and their meaning:
- "^Wil": "Wil" that at the beginning of the text
- "Wil$": "Wil" that at the end of the text

In [ ]:
# 5. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which contain 'Wil' as first three letters for its name.
# Reference: https://www.mongodb.com/docs/manual/reference/operator/query/regex/#mongodb-query-op.-regex
result = collection.aggregate([
    {"$match":{"name": { "$regex": "^Wil"}}},
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("name")
df

In [ ]:
# 6. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which contain 'ces' as last three letters for its name.

result = collection.aggregate([
    {"$match":{"name": {"$regex": "ces$"}}},
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("name")
df

In [ ]:
# 7. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which contain 'Reg' as three letters somewhere in its name.
result = collection.aggregate([
    {"$match":{"name": { "$regex": "Reg"}}}, # You can write as <"$regex": "Reg", "$options": 'i'> so not case sensitive
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("name")
df

In [ ]:
# 8. Write a MongoDB query to find the restaurants which belong to the borough Bronx 
# and prepared either American or Chinese dish
result = collection.aggregate([
    {"$match":
        {
            "borough": "Bronx",
            "$or":
                [
                    {"cuisine" : "American"},
                    {"cuisine" : "Chinese"}
                ]
        }
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0, "grades.grade":1}},
    {"$sort":{"cuisine": 1}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("cuisine")
df


In [ ]:
# 9. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which belong to the borough Staten Island or Queens or Brooklyn.
# Cũng ra đúng kết quả nhưng nhìn quê vl, nên xài cái dưới

result = collection.aggregate([
    {"$match":
        {
            "$or":
                [
                    {"borough" : "Staten Island"},
                    {"borough" : "Queens"},
                    {"borough" : "Bronx"},
                    {"borough" : "Brooklyn"},
                ]
        }
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}},
    {"$sort":{"borough": 1}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("borough")
df

In [ ]:
# 10. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which belong to the borough Staten Island or Queens or Brooklyn.
# Reference: https://www.mongodb.com/docs/manual/reference/operator/query/in/

result = collection.aggregate([
    {"$match":
        {"borough": {"$in" :["Staten Island","Queens","Bronx","Brooklyn"]}}
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("borough")
df

In [ ]:
# 11. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which are not belonging to the borough Staten Island or Queens or Bronx or Brooklyn.
# Reference: https://www.mongodb.com/docs/manual/reference/operator/query/nin/
result = collection.aggregate([
    {"$match":
        {"borough": {"$nin" :["Staten Island","Queens","Bronx","Brooklyn"]}}
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}},
    {"$sort":{"borough": 1}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("borough")
df

In [ ]:
# 12. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which achieved a score which is not more than 10.
result = collection.aggregate([
    {"$match":{"grades.score": {"$lt":10}}},
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "grades.score": 1, "_id":0}},
    {"$sort":{"borough": 1}}
])

pp.pprint(list(result))
# df= pd.DataFrame(result).set_index("borough")
# df


The next task is very interesting:
- If you use \$lt like above, MongoDB will find documents that at least have one key grades.score < 10. Which means documents that have grades.score > 10 will also be counted
- If you use \$not and \$gt, Mongo will do the following:
    + Find all documents that have at least one key grades.score > 10
    + Then take documents that do not belong to the group above.

In [ ]:
# 13. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which achieved a score which is not more than 10.
result = collection.aggregate([
    {"$match": {"grades.score": {"$not": {"$gt":10}}}},
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "grades.score": 1, "_id":0}},
    {"$sort":{"borough": 1}}
])

pp.pprint(list(result))
# df= pd.DataFrame(result).set_index("borough")
# df


In [ ]:
# 14. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which prepared dish except 'American' and 'Chinese' or restaurant's name begins with letter 'Wil'.

result = collection.aggregate([
    {"$match":
        {"$or":
            [
                {"cuisine": {"$nin": ['American', 'Chinese']}},
                {"name": {"$regex": "^Wil"}}
            ]
        }
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}},
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("name")
df

In [ ]:
# 15. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants 
# which prepared dish except 'American' and 'Chinese' or restaurant's name begins with letter 'Wil'.

result = collection.aggregate([
    {"$match":
        {"$or":
            [
                {"name": {"$regex": "^Wil"}},
                {"$and":
                    [
                    {"cuisine": {"$ne":"American"}},
                    {"cuisine": {"$ne":"Chinese"}}
                    ]
                },
            ]
        }
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}}
])

# pp.pprint(list(result))
df= pd.DataFrame(result).set_index("name")
df

In [ ]:
# 16. Write a MongoDB query to find the restaurant Id, name, address and geographical location 
# for those restaurants where 2nd element of coord array contains a value which is more than 42 and upto 52.

# Note: 2nd element thì là coord.1. Hmm. Bây giờ mình muốn address 1st element, 
# đã thử cả <coord>, <coord.0>, <coord.-1>, <coord.2> đều không ra cái gì cả @@
result = collection.aggregate([
    {"$match":
        {"address.coord.1": {"$gt" : 42, "$lte" : 52}}
    },
    {"$project":{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1, "_id":0}},
])

pp.pprint(list(result))
# df= pd.DataFrame(result).set_index("name")
# df